In [1]:
import pandas as pd
import numpy as np
import math
from collections import defaultdict

In [2]:
file_path = 'Excel/sample_excel.xlsx'
df=pd.read_excel(file_path)


In [3]:
df.head()

,roll_no.,year,th
0,1,2024,57
1,2,2024,70
2,3,2024,59
3,4,2024,61
4,5,2024,65


In [4]:
# Assuming 'sample_excel.xlsx' has columns: 'year', 'roll_no.', 'th'
df_Endsem_res = pd.read_excel(file_path, sheet_name="Endsem_res_3y")

max_marks_dict = {
    2022: 80,
    2023: 80,
    2024: 80
}

# --- Calculations for Yearly Data ---

# Create the main summary DataFrame
summary = df_Endsem_res.groupby("year").agg(
    total_students=("roll_no.", "nunique"),
    th_total=("th", "sum")
)

summary["max_marks"] = summary.index.map(max_marks_dict)

# Calculate class average marks for each year
summary["class_avg_marks"] = summary["th_total"] / summary["total_students"]
summary["%_class_avg_marks"] =((summary["class_avg_marks"] / summary["max_marks"]) * 100)

# Calculate Students Who Achieved the Average
df_merged = pd.merge(df_Endsem_res, summary[['class_avg_marks']], on='year', how='left')
achieved_avg_count = df_merged[df_merged['th'] >= df_merged['class_avg_marks']].groupby('year').size()
summary['students_achieved_avg'] = achieved_avg_count
summary['%_students_achieved_avg'] = np.round(
    (summary['students_achieved_avg'] / summary['total_students']) * 100,
    2
)

# Continue with the other calculations
# Calculate Attainment Percentage
summary["attainment_%"] = np.ceil(
    (summary["th_total"] * 100) / (summary["total_students"] * summary["max_marks"])
)
# This column is for the scaled value, initialized with empty values
summary['attainment_scaled_3'] = np.nan


# --- NEW: Calculate and Add Overall Averages for ALL Columns ---

# 1. Calculate the mean of every column from the yearly data
overall_averages = summary.mean()

# 2. The scaled attainment is not a simple mean, so calculate it separately
#    based on the average of the 'attainment_%' column
avg_attainment_scaled = round((overall_averages['attainment_%'] * 3) / 100, 2)
# 3. Add the 'Overall Average' row using the calculated means
summary.loc['Overall Average'] = overall_averages

# 4. Place the specially calculated scaled value into the correct cell
summary.loc['Overall Average', 'attainment_scaled_3'] = avg_attainment_scaled


# --- Display and Export the Final DataFrame ---

print("--- Final Combined Summary DataFrame ---")
# Reordering columns for better readability
summary = summary[[
    'total_students', 
    'th_total', 
    'class_avg_marks', 
    '%_class_avg_marks', 
    'students_achieved_avg', 
    '%_students_achieved_avg', 
    'max_marks', 
    'attainment_%', 
    'attainment_scaled_3'
]]

# Rounding off numerical values for better presentation
summary = summary.round(2)

# Display the final table
summary



--- Final Combined Summary DataFrame ---


,total_students,th_total,class_avg_marks,%_class_avg_marks,students_achieved_avg,%_students_achieved_avg,max_marks,attainment_%,attainment_scaled_3
year,,,,,,,,,
2022,60.0,2596.00,43.27,54.08,31.00,51.67,80.0,55.0,NaN
2023,62.0,2798.00,45.13,56.41,29.00,46.77,80.0,57.0,NaN
2024,70.0,3451.00,49.30,61.62,32.00,45.71,80.0,62.0,NaN
Overall Average,64.0,2948.33,45.90,57.37,30.67,48.05,80.0,58.0,1.74


In [5]:
#sheet 2 Goal set
stud_per_achieved_avg=math.ceil((summary['%_students_achieved_avg']['Overall Average']))
class_per_avg_marks=math.ceil((summary['%_class_avg_marks']['Overall Average']))
data = {
    "level":[1,2,3],
    "% of the student": [stud_per_achieved_avg-5,stud_per_achieved_avg,stud_per_achieved_avg+5],
    "% of marks": [class_per_avg_marks,class_per_avg_marks,class_per_avg_marks]
}

goalset = pd.DataFrame(data)
goalset



,level,% of the student,% of marks
0,1,44,58
1,2,49,58
2,3,54,58


In [21]:

def calculate_attainment_summary(file_path):

    # ============================
    # 1. LOAD MARKS SHEET
    # ============================
    df_tools_1 = pd.read_excel(file_path, sheet_name="IAT+tools")
    tools_marks=pd.read_excel(file_path, sheet_name="IAT+tools")
    # Extract Max Marks row
    max_marks_row = df_tools_1[df_tools_1['Roll_no.'] == 'Max_Marks']
    if max_marks_row.empty:
        raise ValueError("'Max_Marks' row is missing in 'IAT+tools' sheet.")

    max_marks = max_marks_row.iloc[0].to_dict()
    max_marks.pop("Roll_no.")

    # Remove Max Marks Row
    df_tools_1 = df_tools_1[df_tools_1['Roll_no.'] != 'Max_Marks']
    total_students = len(df_tools_1)

    # Convert marks safely
    for col in max_marks:
        df_tools_1[col] = pd.to_numeric(df_tools_1[col], errors="coerce")

    # ============================
    # 2. LOAD TOOL ASSIGNMENT
    # ============================
    tools_df = pd.read_excel(file_path, sheet_name="Tool Assignment")
    tools_assignment=pd.read_excel(file_path, sheet_name="Tool Assignment")
    # ============================
    # 3. LOAD GOALSET (LEVEL TABLE)
    # ============================
    goalset1 = goalset

    # Clean column names (safety)
    goalset1.columns = goalset1.columns.str.strip()

    THRESHOLD = class_per_avg_marks
    output = []

    print("\n--- VALIDATION & DEBUG LOG ---")

    # ============================
    # ATTAINMENT LEVEL FUNCTION
    # ============================
    goalset1 = goalset1.sort_values("level")

    def map_attainment_level(percent_value):
        level = 0
        for _, row in goalset1.iterrows():
            if percent_value >= row["% of the student"]:
                level = row["level"]
        return level

    # ============================
    # 4. PROCESS EACH CO
    # ============================
    for index, row in tools_df.iterrows():

        co_id = row.get("CO-ID")
        co_statement = row.get("CO Statement")

        print(f"\nProcessing CO: {co_id}")

        # Read tools
        t1 = row.get("Tool1")
        t2 = row.get("Tool2")
        t3 = row.get("Tool3")

        # ============================
        # VALIDATION 1 — TOOL ORDER
        # ============================
        if pd.isna(t1) and (not pd.isna(t2) or not pd.isna(t3)):
            print("❌ ERROR: Tool1 is empty but Tool2/Tool3 is filled.")
            print("👉 Fix Excel: Always fill Tool1 first.")
            continue

        if pd.isna(t2) and not pd.isna(t3):
            print("❌ ERROR: Tool2 is empty but Tool3 is filled.")
            print("👉 Fix Excel: Do not skip Tool2.")
            continue

        # ============================
        # VALIDATION 2 — MINIMUM TOOLS
        # ============================
        tools = [t for t in [t1, t2, t3] if not pd.isna(t)]

        if len(tools) < 2:
            print("❌ ERROR: Less than 2 tools provided.")
            print("👉 At least TWO tools are required per CO.")
            continue

        # Extract CO number from ID
        try:
            co_number = int(str(co_id).split(".")[-1])
        except:
            print("❌ ERROR: CO-ID format invalid.")
            print("👉 Expected: format like '2343113.1' or 'CO1'")
            continue

        tool_results = []

        # ============================
        # 5. CALCULATION PER TOOL
        # ============================
        for slot_index, tool_name in enumerate(tools, start=1):

            column_name = f"CO{co_number}_tool_{slot_index}"

            print(f" Mapping: {tool_name} → {column_name}")

            # ============================
            # VALIDATION 3 — COLUMN EXISTS
            # ============================
            if column_name not in df_tools_1.columns:
                print(f"❌ ERROR: Column '{column_name}' NOT FOUND in marks sheet.")
                print("👉 Check column names in 'IAT+tools' sheet.")
                continue

            max_mark = max_marks.get(column_name)

            # ============================
            # VALIDATION 4 — MAX MARKS VALID
            # ============================
            if pd.isna(max_mark) or max_mark == 0:
                print(f"❌ ERROR: Invalid max mark for {column_name}.")
                print("👉 Fix 'Max_Marks' row.")
                continue

            # ============================
            # CALCULATION
            # ============================
            percent_scores = (df_tools_1[column_name] / max_mark) * 100
            achieved = (percent_scores >= THRESHOLD).sum()
            percent = (achieved / total_students) * 100

            tool_results.append({
                "tool": tool_name,
                "students": achieved,
                "percent": round(percent, 2)
            })

        # ============================
        # VALIDATION 5 — BEST TWO SAFETY
        # ============================
        if len(tool_results) < 2:
            print("❌ ERROR: Less than 2 valid tools after validation.")
            print("👉 Check marks sheet & mapping.")
            continue

        # ============================
        # 6. SELECT BEST TWO
        # ============================
        best_two = sorted(tool_results, key=lambda x: x['percent'], reverse=True)[:2]
        best_avg = round((best_two[0]['percent'] + best_two[1]['percent']) / 2, 2)

        # ============================
        # 7. ATTAINMENT LEVEL
        # ============================
        attainment_level = map_attainment_level(best_avg)

        # ============================
        # 8. OUTPUT FORMAT
        # ============================
        row_out = {"CO Statements": co_statement}

        for i, tr in enumerate(tool_results):
            row_out[f"Tool {i+1} Total"] = tr["students"]
            row_out[f"Tool {i+1} %"] = tr["percent"]

        row_out["% considering best 2 tools avg"] = best_avg
        row_out["Attainment Level"] = attainment_level

        output.append(row_out)

    # ============================
    # FINAL TABLE
    # ============================
    print("\n--- PROCESS COMPLETED ---")
    return pd.DataFrame(output),tools_marks,tools_assignment


if __name__ == "__main__":
    direct_attainment_df,tools_marks, tools_assignment = calculate_attainment_summary(file_path)
    print(direct_attainment_df)
    



--- VALIDATION & DEBUG LOG ---

Processing CO: 2343113.1
 Mapping: IAT1 → CO1_tool_1
 Mapping: MCQ → CO1_tool_2
 Mapping: Assignment → CO1_tool_3

Processing CO: 2343113.2
 Mapping: IAT1 → CO2_tool_1
 Mapping: MCQ → CO2_tool_2
 Mapping: Mind Mapping → CO2_tool_3

Processing CO: 2343113.3
 Mapping: IAT1 → CO3_tool_1
 Mapping: MCQ → CO3_tool_2
 Mapping: Assignment → CO3_tool_3

Processing CO: 2343113.4
 Mapping: IAT2 → CO4_tool_1
 Mapping: MCQ → CO4_tool_2
 Mapping: Assignment → CO4_tool_3

Processing CO: 2343113.5
 Mapping: IAT2 → CO5_tool_1
 Mapping: MCQ → CO5_tool_2
 Mapping: Assignment → CO5_tool_3

Processing CO: 2343113.6
 Mapping: IAT2 → CO6_tool_1
 Mapping: MCQ → CO6_tool_2
 Mapping: Assignment → CO6_tool_3

--- PROCESS COMPLETED ---
                                       CO Statements  Tool 1 Total  Tool 1 %  \
0  Students will be able to discuss the need of a...            36     50.00   
1  Students will be able to analyze and design co...            59     81.94   
2  Studen

In [26]:
# -----------------------------------
# LOAD DATA
# -----------------------------------
raw = pd.read_excel(file_path, sheet_name='ESE', header=None)
ESE1=pd.read_excel(file_path, sheet_name='ESE', header=None)
# -----------------------------------
# EXTRACT HEADERS
# -----------------------------------
questions = raw.iloc[0].ffill()
co_map = raw.iloc[1]
max_marks = raw.iloc[2]

# -----------------------------------
# FIND FIRST STUDENT ROW
# -----------------------------------
first_student_row = raw[
    raw.iloc[:, 0].apply(lambda x: str(x).strip().isdigit())
].index[0]

marks_df = raw.iloc[first_student_row:].reset_index(drop=True)

# First column = Roll No
roll_col = marks_df.iloc[:, 0]
marks_only = marks_df.iloc[:, 1:]

# Convert max marks to numeric
max_marks = pd.to_numeric(max_marks[1:], errors="coerce")

# Fix merged headers
questions = questions[1:].ffill()
co_map = co_map[1:]

# -----------------------------------
# PROCESS QUESTION-WISE
# -----------------------------------
results = []

for idx, col in enumerate(marks_only.columns):

    question = questions.iloc[idx]
    coa = co_map.iloc[idx]
    full_marks = max_marks.iloc[idx]

    # -----------------------------------
    # CLEANING LOGIC (IMPORTANT FIX)
    # -----------------------------------
    scores = marks_only[col]

    # Remove whitespace / empty cells
    scores = scores.replace(r'^\s*$', np.nan, regex=True)

    # Convert safely to numeric
    scores = pd.to_numeric(scores, errors="coerce")

    # Drop missing values
    scores = scores.dropna()

    # OPTIONAL: Remove zeros if zero = not attempted
    # scores = scores[scores > 0]

    if len(scores) == 0:
        continue

    # -----------------------------------
    # METRICS (FIXED)
    # -----------------------------------
    attempted = scores.count()

    class_avg = scores.mean()

    # Threshold as % of full marks
    threshold_marks = (full_marks * class_per_avg_marks) / 100

    above_threshold = (scores >= threshold_marks).sum()
    percent_above = (above_threshold / attempted) * 100

    # -----------------------------------
    # STORE
    # -----------------------------------
    results.append({
        "Question": question,
        "CO": coa,
        "Max Marks": full_marks,
        "No. of Students Attempted": attempted,
        "No. >= Threshold": above_threshold,
        "% >= Threshold": round(percent_above, 2),
        "% Class Avg Marks": round(threshold_marks, 2)
    })

# -----------------------------------
# FINAL DATAFRAME (QUESTION-WISE)
# -----------------------------------
ESE_df = pd.DataFrame(results)

print("\n========== QUESTION-WISE ATTAINMENT ==========")
print(ESE_df)

# -----------------------------------
# CO-WISE ATTAINMENT CALCULATION
# -----------------------------------
co_attainment_ese = (
    ESE_df
    .groupby("CO")["% >= Threshold"]
    .mean()
    .reset_index()
)

# Rename column properly
co_attainment_ese.columns = ["CO", "CO Attainment (%)"]

# Round values
co_attainment_ese["CO Attainment (%)"] = co_attainment_ese["CO Attainment (%)"].round(2)

# -----------------------------------
# APPLY ATTAINMENT LEVEL TO CO SUMMARY
# -----------------------------------

goalset1=goalset.sort_values("level")

def map_attainment_level(percent_value):
    level = 0
    for _, row in goalset1.iterrows():
        if percent_value >= row["% of the student"]:
            level = row["level"]
    return level

# Map levels
co_attainment_ese["Attainment Level"] = co_attainment_ese["CO Attainment (%)"].apply(map_attainment_level)


# DISPLAY RESULT
print("\n--- CO-WISE ATTAINMENT WITH LEVEL ---")
co_attainment_ese




========== QUESTION-WISE ATTAINMENT ==========
   Question   CO  Max Marks  No. of Students Attempted  No. >= Threshold  \
0        Q1  CO1          5                         54                47   
1        Q1  CO2          5                         59                53   
2        Q1  CO3          5                         56                45   
3        Q1  CO4          5                         46                39   
4        Q1  CO5          5                         42                36   
5        Q2  CO2         10                         66                57   
6        Q2  CO1          5                         56                25   
7        Q3  CO4         10                         42                18   
8        Q3  CO6          5                         46                29   
9        Q4  CO3         10                         14                 5   
10       Q4  CO1          5                         27                21   
11       Q5  CO5         10             

C:\Users\Sumit Patil\AppData\Local\Temp\ipykernel_18868\1091675636.py:50: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  scores = scores.replace(r'^\s*$', np.nan, regex=True)


,CO,CO Attainment (%),Attainment Level
0,CO1,69.82,3
1,CO2,88.10,3
2,CO3,58.04,3
3,CO4,67.55,3
4,CO5,77.17,3
5,CO6,49.70,2


In [30]:


CES_df = pd.read_excel(file_path, sheet_name="CES")
CES_df_1=pd.read_excel(file_path, sheet_name="CES")
# ============================
# LOAD GOALSET
# ============================
goalset1 = goalset
goalset1.columns = goalset1.columns.str.strip()
goalset1 = goalset1.sort_values("level")

# ============================
# ATTAINMENT LEVEL FUNCTION
# ============================
def map_attainment_level(percent_value):
    level = 0
    for _, row in goalset1.iterrows():
        if percent_value >= row["% of the student"]:
            level = row["level"]
    return level


# ============================
# Step 1: Number of students
# ============================
student_num = CES_df["Sr. No"].nunique()
print("Number of students:", student_num)

# ============================
# Step 2: Create dictionary
# ============================
results = {}

# ============================
# Step 3: Loop through CO columns
# ============================
for co in [c for c in CES_df.columns if c.startswith("CO")]:
    counts = CES_df[co].value_counts()

    # Raw numbers
    num_3 = counts.get(3, 0)
    num_2 = counts.get(2, 0)
    num_1 = counts.get(1, 0)

    # Percentages
    perc_3 = (num_3 * 100) / student_num
    perc_2 = (num_2 * 100) / student_num
    perc_1 = (num_1 * 100) / student_num

    # Weighted overall percentage
    overall_avg = np.round((perc_3 * 3 + perc_2 * 2 + perc_1 * 1) / 3, 2)

    # Goalset mapping
    level = map_attainment_level(overall_avg)

    results[co] = {
        "3's (num)": num_3, "3's (%)": round(perc_3,2),
        "2's (num)": num_2, "2's (%)": round(perc_2,2),
        "1's (num)": num_1, "1's (%)": round(perc_1,2),
        "Overall Average %": overall_avg,
        "Attainment Level": level
    }

# ============================
# Step 4: Final DataFrame
# ============================
indirect_attainment_df = pd.DataFrame(results).T

print("\n--- INDIRECT ATTAINMENT WITH GOALSET LEVEL ---")
indirect_attainment_df


Number of students: 72

--- INDIRECT ATTAINMENT WITH GOALSET LEVEL ---


,3's (num),3's (%),2's (num),2's (%),1's (num),1's (%),Overall Average %,Attainment Level
CO1,44.0,61.11,28.0,38.89,0.0,0.00,87.04,3.0
CO2,37.0,51.39,35.0,48.61,0.0,0.00,83.80,3.0
CO3,38.0,52.78,34.0,47.22,0.0,0.00,84.26,3.0
CO4,40.0,55.56,31.0,43.06,1.0,1.39,84.72,3.0
CO5,38.0,52.78,34.0,47.22,0.0,0.00,84.26,3.0
CO6,39.0,54.17,32.0,44.44,1.0,1.39,84.26,3.0


In [9]:
# ================================
# NBA STYLE MULTI-ROW REPORT
# ================================

# 1) DEFINE MULTI-LEVEL HEADER
multi_columns = pd.MultiIndex.from_tuples([
    ("CO-ID", "", ""),
    # ------- DIRECT ASSESSMENT (90%) -------
    ("Direct Assessment (90%)", "Internal Assessment", "Tool 1"),
    ("Direct Assessment (90%)", "Internal Assessment", "Tool 2"),
    ("Direct Assessment (90%)", "Internal Assessment", "Tool 3"),

    ("Direct Assessment (90%)", "Internal Attainment %", ""),
    ("Direct Assessment (90%)", "Internal Attainment Level", ""),

    ("Direct Assessment (90%)", "End Sem Exam Attainment %", ""),
    ("Direct Assessment (90%)", "End Sem Exam Attainment Level", ""),

    ("Direct Assessment (90%)", "Direct Attainment Level", ""),

    # ------- INDIRECT ASSESSMENT (10%) -------
    ("Indirect Assessment (10%)", "Course Exit Survey", ""),
    ("Indirect Assessment (10%)", "Indirect Attainment Level", ""),

    # ------- FINAL / TARGET / RESULT -------
    ("Final", "Final Attainment Level (Direct 90% + Indirect 10%)", ""),
    ("Target", "CO Attainment Target", ""),
    ("Result", "CO Attainment", "")
])

# 2) CREATE EMPTY REPORT WITH MULTI-HEADERS
final_report_df = pd.DataFrame(index=direct_attainment_df.index, columns=multi_columns)


# 3) MAP VALUES INTO NBA FORMAT
final_report_df[("CO-ID", "", "")]=[f"CO{idx+1}" for idx in range(len(direct_attainment_df))]

# --- Internal Assessment Tools ---
final_report_df[("Direct Assessment (90%)", "Internal Assessment", "Tool 1")] = direct_attainment_df["Tool 1 %"]
final_report_df[("Direct Assessment (90%)", "Internal Assessment", "Tool 2")] = direct_attainment_df["Tool 2 %"]
final_report_df[("Direct Assessment (90%)", "Internal Assessment", "Tool 3")] = direct_attainment_df["Tool 3 %"]

# --- Internal Attainment ---
final_report_df[("Direct Assessment (90%)", "Internal Attainment %", "")] = direct_attainment_df["% considering best 2 tools avg"]
final_report_df[("Direct Assessment (90%)", "Internal Attainment Level", "")] = direct_attainment_df["Attainment Level"]

# --- End Sem Attainment ---
final_report_df[("Direct Assessment (90%)", "End Sem Exam Attainment %", "")] = co_attainment_ese["CO Attainment (%)"]
final_report_df[("Direct Assessment (90%)", "End Sem Exam Attainment Level", "")] = co_attainment_ese["Attainment Level"]

# ---------------------------------------
# DIRECT ATTAINMENT CALCULATION (40% + 60%)
# ---------------------------------------
# Calculate weighted direct attainment percentage
final_report_df[("Direct Assessment (90%)", "Direct Attainment Level", "")] = (
    final_report_df[("Direct Assessment (90%)", "Internal Attainment Level", "")] * 0.4 +
    final_report_df[("Direct Assessment (90%)", "End Sem Exam Attainment Level", "")] * 0.6
)



# --- Indirect ---
final_report_df[("Indirect Assessment (10%)", "Course Exit Survey", "")] = indirect_attainment_df["Overall Average %"].values
final_report_df[("Indirect Assessment (10%)", "Indirect Attainment Level", "")] = indirect_attainment_df["Attainment Level"].values



# --- Final / Target / Result ---
final_report_df[("Final", "Final Attainment Level (Direct 90% + Indirect 10%)", "")] =(
    final_report_df[("Direct Assessment (90%)","Direct Attainment Level", "")] * 0.9 +
    final_report_df[("Indirect Assessment (10%)","Indirect Attainment Level", "" )] * 0.1
)
final_report_df[("Target", "CO Attainment Target", "")] = summary['attainment_scaled_3']['Overall Average']
# -------------------------------
# RESULT: TARGET ACHIEVED OR NOT
# -------------------------------
final_report_df[("Result", "CO Attainment", "")] = np.where(
    final_report_df[("Final", "Final Attainment Level (Direct 90% + Indirect 10%)", "")] >= 
    final_report_df[("Target", "CO Attainment Target", "")],
    "Achieved",
    "Not Achieved"
)


# 4) ROUND
final_report_df = final_report_df.round(2)

# 5) SHOW IN NOTEBOOK
final_report_df


CO-ID Direct Assessment (90%)                                       \
            Internal Assessment                Internal Attainment %   
                         Tool 1 Tool 2  Tool 3                         
0   CO1                   50.00  75.00   97.22                 86.11   
1   CO2                   81.94  81.94  100.00                 90.97   
2   CO3                   36.11  86.11   97.22                 91.66   
3   CO4                   29.17  79.17   97.22                 88.20   
4   CO5                   45.83  72.22   97.22                 84.72   
5   CO6                   80.56  84.72   97.22                 90.97   

                                                       \
  Internal Attainment Level End Sem Exam Attainment %   
                                                        
0                         3                     69.82   
1                         3                     88.10   
2                         3                     58.04   
3                         3                     67.55   
4                         3                     77.17   
5                         3                     49.70   

                                                         \
  End Sem Exam Attainment Level Direct Attainment Level   
                                                          
0                             3                     3.0   
1                             3                     3.0   
2                             3                     3.0   
3                             3                     3.0   
4                             3                     3.0   
5                             2                     2.4   

  Indirect Assessment (10%)                            \
         Course Exit Survey Indirect Attainment Level   
                                                        
0                     87.04                       3.0   
1                     83.80                       3.0   
2                     84.26                       3.0   
3                     84.72                       3.0   
4                     84.26                       3.0   
5                     84.26                       3.0   

                                               Final               Target  \
  Final Attainment Level (Direct 90% + Indirect 10%) CO Attainment Target   
                                                                            
0                                               3.00                 1.74   
1                                               3.00                 1.74   
2                                               3.00                 1.74   
3                                               3.00                 1.74   
4                                               3.00                 1.74   
5                                               2.46                 1.74   

         Result  
  CO Attainment  
                 
0      Achieved  
1      Achieved  
2      Achieved  
3      Achieved  
4      Achieved  
5      Achieved

In [33]:
df_Endsem_res
goalset
tools_marks
tools_assignment
direct_attainment_df
ESE1
co_attainment_ese
CES_df_1
indirect_attainment_df
final_report_df

CO-ID Direct Assessment (90%)                                       \
            Internal Assessment                Internal Attainment %   
                         Tool 1 Tool 2  Tool 3                         
0   CO1                   50.00  75.00   97.22                 86.11   
1   CO2                   81.94  81.94  100.00                 90.97   
2   CO3                   36.11  86.11   97.22                 91.66   
3   CO4                   29.17  79.17   97.22                 88.20   
4   CO5                   45.83  72.22   97.22                 84.72   
5   CO6                   80.56  84.72   97.22                 90.97   

                                                       \
  Internal Attainment Level End Sem Exam Attainment %   
                                                        
0                         3                     69.82   
1                         3                     88.10   
2                         3                     58.04   
3                         3                     67.55   
4                         3                     77.17   
5                         3                     49.70   

                                                         \
  End Sem Exam Attainment Level Direct Attainment Level   
                                                          
0                             3                     3.0   
1                             3                     3.0   
2                             3                     3.0   
3                             3                     3.0   
4                             3                     3.0   
5                             2                     2.4   

  Indirect Assessment (10%)                            \
         Course Exit Survey Indirect Attainment Level   
                                                        
0                     87.04                       3.0   
1                     83.80                       3.0   
2                     84.26                       3.0   
3                     84.72                       3.0   
4                     84.26                       3.0   
5                     84.26                       3.0   

                                               Final               Target  \
  Final Attainment Level (Direct 90% + Indirect 10%) CO Attainment Target   
                                                                            
0                                               3.00                 1.74   
1                                               3.00                 1.74   
2                                               3.00                 1.74   
3                                               3.00                 1.74   
4                                               3.00                 1.74   
5                                               2.46                 1.74   

         Result  
  CO Attainment  
                 
0      Achieved  
1      Achieved  
2      Achieved  
3      Achieved  
4      Achieved  
5      Achieved

In [34]:
!pip install reportlab pillow xlsxwriter

In [37]:
import pandas as pd
import xlsxwriter
from reportlab.lib.pagesizes import landscape, A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Image, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors

# =============================
# FILE PATHS
# =============================
excel_file = "NBA_Full_Attainment_Report.xlsx"
pdf_file   = "NBA_Final_Attainment_Report.pdf"
logo_path  = "university_logo.png"

# =============================
# EXCEL EXPORT + FORMATTING
# =============================
with pd.ExcelWriter(excel_file, engine="xlsxwriter") as writer:

    wb = writer.book

    # =============================
    # SHEETS IN YOUR ORDER
    # =============================
    df_Endsem_res.to_excel(writer, sheet_name="Question Wise ESE", index=False)
    goalset.to_excel(writer, sheet_name="Goalset", index=False)
    tools_marks.to_excel(writer, sheet_name="IAT Tools Marks", index=False)
    tools_assignment.to_excel(writer, sheet_name="Tool Assignment", index=False)
    direct_attainment_df.to_excel(writer, sheet_name="Internal Attainment", index=False)
    ESE1.to_excel(writer, sheet_name="Raw ESE Marks", index=False)
    co_attainment_ese.to_excel(writer, sheet_name="CO Attainment ESE", index=False)
    CES_df_1.to_excel(writer, sheet_name="CES Raw", index=False)
    indirect_attainment_df.to_excel(writer, sheet_name="Indirect Attainment")
    final_report_df.to_excel(writer, sheet_name="Final NBA Report")

    # Select worksheets
    ws_final = writer.sheets["Final NBA Report"]
    ws_ese   = writer.sheets["CO Attainment ESE"]

    # =============================
    # FORMATTING DEFINITIONS
    # =============================
    header = wb.add_format({"bold":True,"border":1,"align":"center","valign":"vcenter"})
    green  = wb.add_format({"bg_color":"#C6EFCE","font_color":"#006100","bold":True})
    red    = wb.add_format({"bg_color":"#FFC7CE","font_color":"#9C0006","bold":True})
    title  = wb.add_format({"bold":True,"font_size":16,"align":"left"})

    # =============================
    # SAFE TITLE PLACEMENT
    # =============================
    ws_final.write("A1", "NBA CO ATTAINMENT REPORT", title)

    # =============================
    # MULTI-ROW HEADER (SHIFTED DOWN)
    # =============================
    for c, col in enumerate(final_report_df.columns):
        ws_final.write(1, c+1, col[1], header)
        ws_final.write(2, c+1, col[2], header)
        ws_final.set_column(c+1, c+1, 18)

    ws_final.set_column(0, 0, 24)

    # =============================
    # CONDITIONAL FORMATTING
    # =============================
    result_col = list(final_report_df.columns).index(("Result","CO Attainment","")) + 1

    ws_final.conditional_format(3, result_col, len(final_report_df)+2, result_col,
        {"type":"text","criteria":"containing","value":"Achieved","format":green})

    ws_final.conditional_format(3, result_col, len(final_report_df)+2, result_col,
        {"type":"text","criteria":"containing","value":"Not Achieved","format":red})

    # =============================
    # CHART 1 — END SEM ATTAINMENT
    # =============================
    chart1 = wb.add_chart({"type":"column"})

    chart1.add_series({
        "name": "CO Attainment %",
        "categories":"='CO Attainment ESE'!$A$2:$A$7",
        "values":"='CO Attainment ESE'!$B$2:$B$7",
        "data_labels":{"value":True}
    })

    chart1.set_title({"name":"End Semester CO Attainment"})
    chart1.set_x_axis({"name":"Course Outcome"})
    chart1.set_y_axis({"name":"% Attainment"})
    ws_ese.insert_chart("E2", chart1, {"x_scale":1.3,"y_scale":1.3})

    # =============================
    # CHART 2 — FINAL vs TARGET
    # =============================
    chart2 = wb.add_chart({"type":"column"})

    final_col  = list(final_report_df.columns).index(("Final","Final Attainment Level (Direct 90% + Indirect 10%)","")) + 1
    target_col = list(final_report_df.columns).index(("Target","CO Attainment Target","")) + 1

    def xlcol(n): return xlsxwriter.utility.xl_col_to_name(n)

    chart2.add_series({
        "name":"Final Attainment",
        "categories":"='Final NBA Report'!$A$4:$A$9",
        "values":f"='Final NBA Report'!${xlcol(final_col)}$4:${xlcol(final_col)}$9",
        "data_labels":{"value":True}
    })

    chart2.add_series({
        "name":"Target",
        "categories":"='Final NBA Report'!$A$4:$A$9",
        "values":f"='Final NBA Report'!${xlcol(target_col)}$4:${xlcol(target_col)}$9",
        "data_labels":{"value":True}
    })

    chart2.set_title({"name":"Final Attainment vs Target"})
    chart2.set_x_axis({"name":"CO"})
    chart2.set_y_axis({"name":"Level"})
    ws_final.insert_chart("B15", chart2, {"x_scale":1.3,"y_scale":1.3})

# =============================
# PDF EXPORT
# =============================
styles = getSampleStyleSheet()
doc = SimpleDocTemplate(pdf_file, pagesize=landscape(A4))
content = []

content.append(Paragraph("NBA CO ATTAINMENT REPORT", styles["Title"]))
content.append(Spacer(1,15))

try:
    img = Image(logo_path, width=130, height=60)
    content.append(img)
    content.append(Spacer(1,10))
except:
    pass

pdf_data = [[""] + list(final_report_df.columns.get_level_values(1))]
for i, row in final_report_df.iterrows():
    pdf_data.append([i] + list(row.values))

table = Table(pdf_data)
table.setStyle(TableStyle([
    ("GRID",(0,0),(-1,-1),1,colors.black),
    ("BACKGROUND",(0,0),(-1,0),colors.lightgrey),
    ("ALIGN",(0,0),(-1,-1),"CENTER"),
    ("FONT",(0,0),(-1,0),"Helvetica-Bold"),
]))

content.append(table)
doc.build(content)

print("✅ Excel File Created:", excel_file)
print("✅ PDF File Created:", pdf_file)


✅ Excel File Created: NBA_Full_Attainment_Report.xlsx
✅ PDF File Created: NBA_Final_Attainment_Report.pdf
